# Predspracovanie dát

In [30]:
import sys
import os
from pyspark.sql.functions import when, count, col
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

from integration.integration import integration
from pyspark.sql import SparkSession
from collections import Counter

In [31]:
spark = SparkSession.builder.appName("preprocessing").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [32]:
df_train, df_test = integration()
df_train = df_train.drop("Accident_Index", "Date")
df_test = df_test.drop("Accident_Index","Date")

#### Zistenie, ktoré stĺpce sú kvantitatívne a kategorické

In [33]:
types = df_train.dtypes
print(types)

[('Location_Easting_OSGR', 'int'), ('Location_Northing_OSGR', 'int'), ('Longitude', 'double'), ('Latitude', 'double'), ('Police_Force', 'int'), ('Accident_Severity', 'int'), ('Number_of_Vehicles', 'int'), ('Number_of_Casualties', 'int'), ('Day_of_Week', 'int'), ('Time', 'timestamp'), ('Local_Authority_(District)', 'int'), ('Local_Authority_(Highway)', 'string'), ('1st_Road_Class', 'int'), ('1st_Road_Number', 'int'), ('Road_Type', 'int'), ('Speed_limit', 'int'), ('Junction_Detail', 'int'), ('Junction_Control', 'int'), ('2nd_Road_Class', 'int'), ('2nd_Road_Number', 'int'), ('Pedestrian_Crossing-Human_Control', 'int'), ('Pedestrian_Crossing-Physical_Facilities', 'int'), ('Light_Conditions', 'int'), ('Weather_Conditions', 'int'), ('Road_Surface_Conditions', 'int'), ('Special_Conditions_at_Site', 'int'), ('Carriageway_Hazards', 'int'), ('Urban_or_Rural_Area', 'int'), ('Did_Police_Officer_Attend_Scene_of_Accident', 'int'), ('LSOA_of_Accident_Location', 'string'), ('Vehicle_Reference', 'int')

In [34]:
counts = Counter()

for dtype in types:
    counts[dtype[1]] += 1

print(counts)

Counter({'int': 59, 'double': 2, 'string': 2, 'timestamp': 1})


#### Vypočítanie základných štatistík pre kvantitatívne stĺpce

In [35]:
def get_int(df):
    int_columns = []

    for column, data_type in df.dtypes:
        if data_type == "int":
            int_columns.append(column)
    return int_columns

In [36]:
def show_stats(df, columns, batch_size=4):
    for i in range(0, len(columns), batch_size):
        df.describe(columns[i:i+batch_size]).show()

show_stats(df_train, get_int(df_train))

+-------+---------------------+----------------------+------------------+------------------+
|summary|Location_Easting_OSGR|Location_Northing_OSGR|      Police_Force| Accident_Severity|
+-------+---------------------+----------------------+------------------+------------------+
|  count|               300276|                300276|            300293|            300293|
|   mean|      438255.11356885|    300105.18741757586| 30.78688480916971|2.8231294102759636|
| stddev|    94748.83907367043|    157397.67714086492|24.743146328252084|0.4288013609691326|
|    min|                65860|                 10560|                 1|                 1|
|    max|               655370|               1191500|                98|                 3|
+-------+---------------------+----------------------+------------------+------------------+



+-------+------------------+--------------------+------------------+--------------------------+
|summary|Number_of_Vehicles|Number_of_Casualties|       Day_of_Week|Local_Authority_(District)|
+-------+------------------+--------------------+------------------+--------------------------+
|  count|            300293|              300293|            300293|                    300293|
|   mean|2.3571978034786025|   2.121068423173367| 4.110751832377045|          355.383961664108|
| stddev|  2.50916130396624|  3.2845832805738686|1.9431281459177299|        252.13048697950327|
|    min|                 1|                   1|                 1|                         1|
|    max|                67|                  93|                 7|                       941|
+-------+------------------+--------------------+------------------+--------------------------+



+-------+------------------+------------------+------------------+------------------+
|summary|    1st_Road_Class|   1st_Road_Number|         Road_Type|       Speed_limit|
+-------+------------------+------------------+------------------+------------------+
|  count|            300293|            300293|            300293|            300293|
|   mean| 3.912422200983706| 998.2143373305405|5.0935453040863425| 41.09957608069452|
| stddev|1.4508332728073978|1808.0544016265796| 1.647924197132397|15.104004572455908|
|    min|                 1|                -1|                 1|                10|
|    max|                 6|              9999|                 9|                70|
+-------+------------------+------------------+------------------+------------------+



+-------+------------------+------------------+-----------------+------------------+
|summary|   Junction_Detail|  Junction_Control|   2nd_Road_Class|   2nd_Road_Number|
+-------+------------------+------------------+-----------------+------------------+
|  count|            300293|            300293|           300293|            300293|
|   mean|2.3728924750160676|1.7908076445338386|2.622718478286207| 386.6823202672057|
| stddev|2.6134518815478947| 2.303394959580854|3.211294477117478|1312.3022275267308|
|    min|                -1|                -1|               -1|                -1|
|    max|                 9|                 4|                6|              9999|
+-------+------------------+------------------+-----------------+------------------+



+-------+---------------------------------+---------------------------------------+------------------+------------------+
|summary|Pedestrian_Crossing-Human_Control|Pedestrian_Crossing-Physical_Facilities|  Light_Conditions|Weather_Conditions|
+-------+---------------------------------+---------------------------------------+------------------+------------------+
|  count|                           300293|                                 300293|            300293|            300293|
|   mean|             0.008082106476008431|                     0.6646475275813956| 1.929049295188366|1.5706992836995868|
| stddev|              0.11848045890628368|                     1.7475343894531297|1.6382984972923809|1.6184739593558908|
|    min|                               -1|                                     -1|                 1|                -1|
|    max|                                2|                                      8|                 7|                 9|
+-------+---------------

...

[2205.131s][warning][gc,alloc] Executor task launch worker for task 0.0 in stage 940.0 (TID 2409): Retried waiting for GCLocker too often allocating 1048578 words


+-------+-------------------+----------------------------+-------------------------+------------------+
|summary|1st_Point_of_Impact|Was_Vehicle_Left_Hand_Drive?|Journey_Purpose_of_Driver|     Sex_of_Driver|
+-------+-------------------+----------------------------+-------------------------+------------------+
|  count|             300293|                      300293|                   300293|            300293|
|   mean| 1.7344160536542643|          0.9889507913937388|        8.837645233155618|1.3894696180064137|
| stddev| 1.1190713499544918|         0.18852904383972255|        5.980739624197323|0.5807067891130099|
|    min|                 -1|                          -1|                       -1|                -1|
|    max|                  4|                           2|                       15|                 3|
+-------+-------------------+----------------------------+-------------------------+------------------+



[2217.940s][warning][gc,alloc] Executor task launch worker for task 2.0 in stage 957.0 (TID 2454): Retried waiting for GCLocker too often allocating 1048578 words


+-------+------------------+------------------+--------------------+------------------+
|summary|     Age_of_Driver|Age_Band_of_Driver|Engine_Capacity_(CC)|   Propulsion_Code|
+-------+------------------+------------------+--------------------+------------------+
|  count|            300293|            300293|              300293|            300293|
|   mean| 34.95248973502546| 5.968504094334533|    1467.86069272344|0.8100921433400046|
| stddev|19.064810411524313| 2.815962035850567|  1678.0161198258568|1.1314741218330875|
|    min|                -1|                -1|                  -1|                -1|
|    max|                99|                11|               99999|                12|
+-------+------------------+------------------+--------------------+------------------+



+-------+-----------------+------------------+---------------------+
|summary|   Age_of_Vehicle| Driver_IMD_Decile|Driver_Home_Area_Type|
+-------+-----------------+------------------+---------------------+
|  count|           300293|            300293|               300293|
|   mean|5.011478789049362| 3.630224480757127|   0.9206941220741076|
| stddev|5.294180187151661|3.7267864561304447|   1.1077424779630716|
|    min|               -1|                -1|                   -1|
|    max|              110|                10|                    3|
+-------+-----------------+------------------+---------------------+



#### Zobrazenie histogramu pre kategorické atribúty

In [37]:
df_train.groupby("Local_Authority_(Highway)").count().orderBy("count", ascending=False).show()

+-------------------------+-----+
|Local_Authority_(Highway)|count|
+-------------------------+-----+
|                E10000016| 9067|
|                E10000030| 8536|
|                E10000017| 7218|
|                E10000012| 6495|
|                E10000015| 6219|
|                E08000025| 5805|
|                E10000014| 5748|
|                E10000028| 5051|
|                E10000019| 4289|
|                E08000035| 4230|
|                E10000024| 4218|
|                E10000007| 4170|
|                E10000020| 3831|
|                E10000032| 3822|
|                E10000023| 3772|
|                E10000008| 3671|
|                E10000029| 3628|
|                E10000003| 3550|
|                E10000025| 3258|
|                E08000032| 3116|
+-------------------------+-----+
only showing top 20 rows



In [38]:
df_train.groupby("LSOA_of_Accident_Location").count().orderBy("count", ascending=False).show()

+-------------------------+-----+
|LSOA_of_Accident_Location|count|
+-------------------------+-----+
|                     NULL|19875|
|                E01024597|  335|
|                E01000004|  308|
|                E01018648|  252|
|                E01023722|  251|
|                E01011365|  244|
|                E01023584|  211|
|                E01030458|  190|
|                E01023725|  170|
|                E01004736|  164|
|                E01005131|  160|
|                E01018465|  154|
|                E01016481|  153|
|                E01002444|  152|
|                E01029317|  148|
|                E01023732|  135|
|                E01018337|  130|
|                E01006650|  128|
|                E01017620|  128|
|                E01008440|  127|
+-------------------------+-----+
only showing top 20 rows



#### Chýbajúce hodnoty

In [39]:
df_train.select([count(when(col(c).isNull(), c)).alias(c) for c in df_train.columns]).show()

+---------------------+----------------------+---------+--------+------------+-----------------+------------------+--------------------+-----------+----+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+-----------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+-------------+----------------------+----------------------------------+-------------+-----------------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+--

# TODO:
- [ ] Informačný zisk - F
- [ ] Nahradiť chýbajúce hodnoty - M
- [ ] Nominálne atribúty na kvantitatívne - M
- [ ] Kvantitívne atribúty na nominálne (neviem prečo) - F
- [ ] Normalizácia - StandardScaler - F
- [ ] One-hot encoding - M
- [ ] Odstrániť irelevantné atribúty - J